<a href="https://colab.research.google.com/github/osaeed-ds/cass-vector/blob/main/Cassandra_DSE_Vector_Search_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with this notebook

- Create a DSE 7 or Cassandra 5 database and have your login, password, and IP
- To generate embeddings, we will use Sentence Transformers from LangChain configured with the all-MiniLM-L6-v2 model from Hugging Face


# Setup

In [17]:
!pip install openai datasets pandas cassandra-driver sentence-transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=96e9fefd3c2b9ba986b8ebb23d2430ad35d1effae609cf29ccb680b1954f79d2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


# Imports

In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
import numpy
import pandas as pd

# Keys & Environment Variables

In [12]:
# keys and tokens here
cass_user = 'cassandra'
cass_pw = 'cassandra'
cass_ip = '18.222.204.232'
cass_keyspace ='vector_preview'

# Select a model to compute embeddings

In [18]:
# Download the all-MiniLM-L6-v2 from HuggingFace, and configure it to infer on the CPU (as opposed to GPU)
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')

(…)f3d3c277d6e90027e55de9125/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)f80f3d3c277d6e90027e55de9125/config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

(…)de9125/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)f3d3c277d6e90027e55de9125/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

(…)3d3c277d6e90027e55de9125/train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)80f3d3c277d6e90027e55de9125/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

# Connect to the Cluster

In [6]:
# Don't mind the "Closing connection" error after "downgrading protocol..." messages,
# it is really just a warning: the connection will work smoothly.
auth_provider = PlainTextAuthProvider(cass_user, cass_pw)
cluster = Cluster(
    [cass_ip],
    auth_provider=auth_provider
)

session = cluster.connect()
keyspace = cass_keyspace

# Drop / Create Schema

In [7]:
# only use this to reset the schema
session.execute("DROP INDEX IF EXISTS {}.openai_desc".format(cass_keyspace))
session.execute("DROP INDEX IF EXISTS {}.minilm_desc".format(cass_keyspace))
session.execute("DROP TABLE IF EXISTS {}.products_table".format(cass_keyspace))

In [8]:

# # Create Table
session.execute("""CREATE TABLE IF NOT EXISTS {}.products_table
(product_id int,
 chunk_id int,

 product_name text,
 description text,
 price text,

 openai_description_embedding vector<float, 1536>,
 minilm_description_embedding vector<float, 384>,

 PRIMARY KEY (product_id, chunk_id))""".format(cass_keyspace))

# # Create Index
session.execute("""CREATE CUSTOM INDEX IF NOT EXISTS openai_desc ON {}.products_table (openai_description_embedding) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'""".format(cass_keyspace))
session.execute("""CREATE CUSTOM INDEX IF NOT EXISTS minilm_desc ON {}.products_table (minilm_description_embedding) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'""".format(cass_keyspace))


They don't yet support SELECT queries with:
 * Consistency level higher than ONE/LOCAL_ONE.
 * Paging.
 * No LIMIT clauses.
 * PER PARTITION LIMIT clauses.
 * GROUP BY clauses.
 * Aggregation functions.
 * Filters on columns without a SAI index.
They don't yet support SELECT queries with:
 * Consistency level higher than ONE/LOCAL_ONE.
 * Paging.
 * No LIMIT clauses.
 * PER PARTITION LIMIT clauses.
 * GROUP BY clauses.
 * Aggregation functions.
 * Filters on columns without a SAI index.
They don't yet support SELECT queries with:
 * Consistency level higher than ONE/LOCAL_ONE.
 * Paging.
 * No LIMIT clauses.
 * PER PARTITION LIMIT clauses.
 * GROUP BY clauses.
 * Aggregation functions.
 * Filters on columns without a SAI index.


# Load the table with data and create text embeddings



In [9]:
import datasets
from datasets import load_dataset
dataset = load_dataset("jdabello/products", split="train")
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['product_id', 'product_name', 'description', 'price'],
    num_rows: 1081
})

In [10]:
products_list = dataset.to_pandas()
products_list

,product_id,product_name,description,price
0,552,Sony Turntable - PSLX350H,Sony Turntable - PSLX350H/ Belt Drive System/ ...,None
1,580,Bose Acoustimass 5 Series III Speaker System -...,Bose Acoustimass 5 Series III Speaker System -...,$399.00
2,4696,Sony Switcher - SBV40S,Sony Switcher - SBV40S/ Eliminates Disconnecti...,$49.00
3,5644,Sony 5 Disc CD Player - CDPCE375,Sony 5 Disc CD Player- CDPCE375/ 5 Disc Change...,None
4,6284,Bose 27028 161 Bookshelf Pair Speakers In Whit...,Bose 161 Bookshelf Speakers In White - 161WH/ ...,$158.00
...,...,...,...,...
1076,39088,Logitech Cordless Desktop Wave Keyboard And Mo...,Logitech Cordless Desktop Wave Keyboard And Mo...,$79.00
1077,39090,Mitsubishi DLP Black TV Stand - MBS73V,Mitsubishi DLP Black TV Stand - MBS73V/ Matchi...,$549.00
1078,39175,Logitech Digital Precision PC Gaming Headset -...,Logitech Digital Precision PC Gaming Headset -...,$49.00
1079,39176,Logitech 2.1 Multimedia Silver Speaker System ...,Logitech 2.1 Multimedia Silver Speaker System ...,None


In [20]:

for id, row in products_list.iterrows():
  # Create Embedding for each conversation row, save them to the database
  text_chunk_length = 2500
  text_chunks = [row.description[i:i + text_chunk_length] for i in range(0, len(row.description), text_chunk_length)]
  for chunk_id, chunk in enumerate(text_chunks):
    pricevalue = row.price if isinstance(row.price, str) else ""
    full_chunk = f"{chunk} price: {pricevalue}"
    embedding = model.encode(full_chunk).tolist()
    query = SimpleStatement(
                """
                INSERT INTO {}.products_table
                (product_id, chunk_id, product_name, description, price, minilm_description_embedding)
                VALUES (%s, %s, %s, %s, %s, %s)
                """.format(cass_keyspace)
            )
    display(row)

    session.execute(query, (row.product_id, chunk_id, row.product_name, row.description, pricevalue, embedding))

product_id                                                    552
product_name                            Sony Turntable - PSLX350H
description     Sony Turntable - PSLX350H/ Belt Drive System/ ...
price                                                        None
Name: 0, dtype: object

product_id                                                    580
product_name    Bose Acoustimass 5 Series III Speaker System -...
description     Bose Acoustimass 5 Series III Speaker System -...
price                                                     $399.00
Name: 1, dtype: object

product_id                                                   4696
product_name                               Sony Switcher - SBV40S
description     Sony Switcher - SBV40S/ Eliminates Disconnecti...
price                                                      $49.00
Name: 2, dtype: object

product_id                                                   5644
product_name                     Sony 5 Disc CD Player - CDPCE375
description     Sony 5 Disc CD Player- CDPCE375/ 5 Disc Change...
price                                                        None
Name: 3, dtype: object

product_id                                                   6284
product_name    Bose 27028 161 Bookshelf Pair Speakers In Whit...
description     Bose 161 Bookshelf Speakers In White - 161WH/ ...
price                                                     $158.00
Name: 4, dtype: object

...

product_id                                                  39088
product_name    Logitech Cordless Desktop Wave Keyboard And Mo...
description     Logitech Cordless Desktop Wave Keyboard And Mo...
price                                                      $79.00
Name: 1076, dtype: object

product_id                                                  39090
product_name               Mitsubishi DLP Black TV Stand - MBS73V
description     Mitsubishi DLP Black TV Stand - MBS73V/ Matchi...
price                                                     $549.00
Name: 1077, dtype: object

product_id                                                  39175
product_name    Logitech Digital Precision PC Gaming Headset -...
description     Logitech Digital Precision PC Gaming Headset -...
price                                                      $49.00
Name: 1078, dtype: object

product_id                                                  39176
product_name    Logitech 2.1 Multimedia Silver Speaker System ...
description     Logitech 2.1 Multimedia Silver Speaker System ...
price                                                        None
Name: 1079, dtype: object

product_id                                                  39179
product_name           Linksys Wireless-G Range Expander - WRE54G
description     Linksys Wireless-G Range Expander - WRE54G/ Su...
price                                                      $99.00
Name: 1080, dtype: object



---


# Start using the index

In the steps up to this point, we have been creating a schema and loading the table with data, including embeddings we generated through the Sentence Transformer Embedding.


# Convert a query string into a text embedding to use as part of the query

This is where the real fun starts.  Provide a question or request to be used as the query.  The source sample database is mostly consumer electronics and appliances, so imagine you're talking to a customer service rep at an electronics store.

Here we use the same API that we used to calculate embeddings for each row in the database, but this time we are using your input question to calculate a vector to use in a query.

In [21]:
customer_input = "What equipement would you recommend for a computer workstation setup costing less than $2000?"

embedding = model.encode(customer_input).tolist()
display(embedding)

[0.002266432624310255,
 0.013617303222417831,
 0.027773922309279442,
 -0.0174216590821743,
 -0.012639481574296951,
 -0.014180080965161324,
 -0.03305533528327942,
 0.0796174556016922,
 -0.007120377849787474,
 0.020291859284043312,
 -0.04019630700349808,
 -0.01538899540901184,
 0.030922377482056618,
 0.011547531001269817,
 -0.022810542955994606,
 0.050305262207984924,
 0.052210401743650436,
 -0.051781538873910904,
 0.090920589864254,
 -0.05670025944709778,
 -0.014529856853187084,
 -0.043305572122335434,
 -0.0462927483022213,
 -0.03880772367119789,
 0.04814833402633667,
 0.02292833849787712,
 0.10044553130865097,
 -0.008148678578436375,
 -0.016650674864649773,
 0.03752529248595238,
 -0.054889827966690063,
 0.0026361073832958937,
 0.004633632954210043,
 -0.028779814019799232,
 -0.0026259757578372955,
 -0.0207726638764143,
 0.11245278269052505,
 0.01207745261490345,
 -0.054619476199150085,
 -0.08424900472164154,
 -0.00041691630030982196,
 0.030957333743572235,
 -0.05956203490495682,
 0.0451

Let's take a look at what a query against a vector index could look like.  The query vector has the same dimensions (number of entries in the list) as the embeddings we generated a few steps ago for each row in the database.

In [22]:
query = SimpleStatement(
    """
    SELECT *
    FROM {}.products_table
    ORDER BY minilm_description_embedding ANN OF {} LIMIT 5;
    """.format(cass_keyspace,embedding)
    )
display(query)

<SimpleStatement query="
    SELECT *
    FROM vector_preview.products_table
    ORDER BY minilm_description_embedding ANN OF [0.002266432624310255, 0.013617303222417831, 0.027773922309279442, -0.0174216590821743, -0.012639481574296951, -0.014180080965161324, -0.03305533528327942, 0.0796174556016922, -0.007120377849787474, 0.020291859284043312, -0.04019630700349808, -0.01538899540901184, 0.030922377482056618, 0.011547531001269817, -0.022810542955994606, 0.050305262207984924, 0.052210401743650436, -0.051781538873910904, 0.090920589864254, -0.05670025944709778, -0.014529856853187084, -0.043305572122335434, -0.0462927483022213, -0.03880772367119789, 0.04814833402633667, 0.02292833849787712, 0.10044553130865097, -0.008148678578436375, -0.016650674864649773, 0.03752529248595238, -0.054889827966690063, 0.0026361073832958937, 0.004633632954210043, -0.028779814019799232, -0.0026259757578372955, -0.0207726638764143, 0.11245278269052505, 0.01207745261490345, -0.054619476199150085, -0.08424900472

# Find the top 5 results using ANN Similarity

In [23]:
results = session.execute(query)
top_5_products = results._current_rows

for row in top_5_products:
  print(f"""{row.product_id}, {row.product_name}, {row.description}, {row.price}\n""")

34697, LaCie 500GB d2 Quadra External Hard Drive - 301825U, LaCie 500GB d2 Quadra External Hard Drive - 301825U/ Quadruple Interface For Full PC And Mac Compatibility/ Interface Bandwidth Up To 3Gbits/s (eSATA)/ Advanced Aluminum Heat Sink Design Cooling System For Quiet Operation/ 7200 Rotational Speed (rpm)/ 16MB Cache/ Compatible With Time Machine, $159.00

37811, Sony VAIO LV Series Silver All-In-One Desktop Computer - VGCLV150J, Sony VAIO LV Series Silver All-In-One Desktop Computer - VGCLV150J/ 2.53GHz Intel Core 2 Duo E7200 Processor/ 4GB Memory/ Built-In 1.3 MP MOTION EYE Camera And Microphone/ 500GB Serial ATA Hard Drive/ 24' XBRITE-Full HD LCD/ Integrated Stereo A2DP Bluetooth Technology/ Two Built-In 5.5 Watt Stereo Speakers/ Microsoft Windows Vista Home Premium 64-Bit/ Silver Finish, $1,899.00

37293, Sony VAIO AW Series Titanium Grey Notebook Computer - VGNAW110JH, Sony VAIO AW Series Titanium Grey Notebook Computer - VGNAW110JH/ 2.26GHz Intel Core 2 Duo Processor P8400/ 1